In [ ]:
import sys, os, subprocess, shutil, glob, pathlib

print("Python:", sys.version)
print("Interpreter:", sys.executable)

env_name = os.path.basename(sys.prefix) or "current-env"
print("Detected environment name:", env_name)

req_file = "requirements.txt"
if os.path.exists(req_file):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", req_file])
else:
    subprocess.check_call([sys.executable, "-m", "pip", "install",
                           "numpy==1.23.5", "scipy==1.15.3",
                           "parmap==1.7.0", "pandas==2.3.1",
                           "scikit-learn==1.7.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "setuptools>=68", "wheel", "Cython<3.1"])
subprocess.check_call([sys.executable, "-m", "ipykernel", "install",
                       "--user", "--name", env_name, "--display-name", f"Python ({env_name})"])

for p in ["build", "__pycache__"]:
    shutil.rmtree(p, ignore_errors=True)
for f in glob.glob("ns_hmm.*.so") + glob.glob("ns_hmm.*.pyd") + glob.glob("ns_hmm.c"):
    pathlib.Path(f).unlink(missing_ok=True)

for key in ("CFLAGS", "CPPFLAGS", "CXXFLAGS"):
    if key in os.environ:
        os.environ.pop(key)

if os.path.exists("setup.py"):
    print("\n[Build] python setup.py build_ext --inplace ...")
    subprocess.check_call([sys.executable, "setup.py", "build_ext", "--inplace"])
else:
    raise FileNotFoundError("setup.py not found in current directory.")

import importlib
try:
    import ns_hmm
except Exception:
    if "ns_hmm" in sys.modules:
        del sys.modules["ns_hmm"]
    importlib.invalidate_caches()
    import ns_hmm

print("\nSetup complete. ns_hmm loaded:", ns_hmm)
print(f"Jupyter kernel registered as: Python ({env_name})")


In [ ]:
import numpy as np
import scipy as sp
import matplotlib
from matplotlib import pyplot as plt
import math
import seaborn as sns
from matplotlib import gridspec
import multiprocessing
from multiprocessing import get_context
import parmap
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator
from sklearn.mixture import BayesianGaussianMixture as BGMM
import sys, subprocess, shutil, glob, pathlib, os, site, platform, threading
import pickle
from datetime import datetime
print("Python:", sys.version)
print("Interpreter:", sys.executable)
print("Site-packages:", site.getsitepackages())
print("OS:", platform.platform())

In [ ]:
plt.rcParams["font.family"] = "times new roman"
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams['figure.dpi'] = 75
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20 

from matplotlib_inline.backend_inline import set_matplotlib_formats

# Nested sampling run

In [ ]:
# user-configurable parameters
user_params = {
    "D_min": 0.001,
    "D_max": 2.0,
    "unit_T": 0.1,
    "unit_L": 0.167,
}
path = 'test_dataset'

ns_hmm.set_params(user_params)
fnames = np.genfromtxt(f'{path}/filenames.txt', dtype=str)
print(ns_hmm.get_params())  # check

fnum = len(fnames)
findices = np.arange(1, fnum + 1)
num_cores = min(os.cpu_count(), fnum)
indices = np.array_split(findices, num_cores)

print(fnames)
for i in range(fnum):
    globals()[f'trj_{i+1}'] = np.genfromtxt(f'{path}/{fnames[i]}')


In [ ]:
def start_logger(ctx):
    mgr = ctx.Manager()
    q = mgr.Queue()
    def _listener():
        for msg in iter(q.get, None):
            print(msg, flush=True)
    t = threading.Thread(target=_listener, daemon=True)
    t.start()
    return q, t, mgr

start_method = "fork" if sys.platform != "win32" else "spawn"
ctx = get_context(start_method)

USE_LOG_QUEUE = True
log_q, t, mgr = (None, None, None)
if USE_LOG_QUEUE:
    log_q, t, mgr = start_logger(ctx)

def _init_worker(params):
    import ns_hmm as _m
    _m.set_params(params)

with ctx.Pool(processes=num_cores, initializer=_init_worker, initargs=(user_params,)) as pool:
    results = parmap.map(
        ns_hmm.mp_nested_sampling,
        indices,
        200, 100000, 0.0001, 0.25, 30, path,
        log_q=log_q,
        pm_pbar=True, pm_pool=pool
    )

if USE_LOG_QUEUE:
    log_q.put(None)
    t.join()


# AIC calculation

In [ ]:
AIC = np.zeros((fnum, 3))
for j in range(fnum):
    temp_AIC = np.inf
    for k in range(3):
        nstates = k + 1
        with open('{}/results/{}totalseries{}state.pkl'.format(path, j+1, nstates), 'rb') as f:
            temp = pickle.load(f)
        temp_MLE = temp[np.argmax(temp[:, nstates**2]), :]
        AIC[j, k] = 2 * (nstates ** 2) - 2 * temp_MLE[nstates ** 2]
        
        if AIC[j, k] < temp_AIC: # model selection
            temp_AIC = AIC[j, k]
            globals()['MLE{}'.format(j+1)] = temp_MLE.copy() # MLE with minimum AIC

with open('{}/results/AIC.pkl'.format(path), 'wb') as f:
    pickle.dump(AIC, f)
        

# Model selection

In [ ]:
model_AIC = np.zeros(fnum, dtype = int)
with open('{}/results/AIC.pkl'.format(path), 'rb') as f:
    IC = pickle.load(f)
model_AIC = np.argmin(IC, axis = 1) + 1
            
model_Bayes = np.zeros(fnum, dtype = int)
for i in range(fnum):
    with open('{}/results/prob{}.pkl'.format(path, i + 1), 'rb') as f:
        prob = pickle.load(f)
    model_Bayes[i] = np.argmax(prob) + 1

# Parameter estimation (MLE, MAP)

In [ ]:
ns_hmm.NS_Params('uniform', 'AIC', path, fnum)
ns_hmm.NS_Params('uniform', 'Bayesian', path, fnum)

In [ ]:
AIC_MLE_ds = []
AIC_MAP_ds = []

####--AIC, MLE

for j in range(fnum):
    with open('{}/results/MLE{}_{}.pkl'.format(path, j+1, 'AIC'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        AIC_MLE_ds.append(temp[k])
        
####--AIC, MLE

####--AIC, MAP

for j in range(fnum):
    with open('{}/results/MAP{}_{}.pkl'.format(path, j+1, 'AIC'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        AIC_MAP_ds.append(temp[k])

####--AIC, MAP

Bayes_MLE_ds = []
Bayes_MAP_ds = []

####--Bayes, MLE

for j in range(fnum):
    with open('{}/results/MLE{}_{}.pkl'.format(path, j+1, 'Bayesian'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        Bayes_MLE_ds.append(temp[k])
        
####--Bayes, MLE


####--Bayes, MAP

for j in range(fnum):
    with open('{}/results/MAP{}_{}.pkl'.format(path, j+1, 'Bayesian'), 'rb') as f:
        temp = pickle.load(f)
    temp = temp.flatten()
    for k in range(np.sqrt(len(temp)-1).astype(int)):
        Bayes_MAP_ds.append(temp[k])

####--Bayes, MAP

# Parameter clustering

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from sklearn.mixture import BayesianGaussianMixture as BGMM
import pickle

# ===== helpers =====
def _prep_log10_pos(arr, name):
    arr = np.asarray(arr, float).ravel()
    arr = arr[np.isfinite(arr) & (arr > 0)]
    if arr.size == 0:
        raise ValueError(f"{name}")
    return np.log10(arr)

def _safe_hist(log10_vals, bins_lin):
    hist, bins = np.histogram(log10_vals, density=True, bins=bins_lin)
    w = bins[1] - bins[0]
    centers = bins[:-1] + w/2
    return hist, centers, w

def _seg_max(hist, i0=None, i1=None):
    h = hist if (i0 is None and i1 is None) else hist[i0:i1]
    return 0.0 if h.size == 0 else float(np.max(h))

def _quad_boundary(mu1, var1, w1, mu2, var2, w2, eps=1e-12):
    var1 = float(max(var1, eps))
    var2 = float(max(var2, eps))

    A = (1.0/var2) - (1.0/var1)
    B = 2.0*(mu1/var1 - mu2/var2)
    C = (mu2*mu2)/var2 - (mu1*mu1)/var1 + 2.0*np.log(max(w1, eps)/max(w2, eps)) - np.log(var1/var2)

    if abs(A) < 1e-14:
        if abs(B) < 1e-14:
            return 0.5*(mu1+mu2)
        x = -C/B
        return x

    disc = B*B - 4.0*A*C
    if disc < 0:
        return 0.5*(mu1+mu2)

    sqrt_disc = np.sqrt(disc)
    x1 = (-B + sqrt_disc) / (2.0*A)
    x2 = (-B - sqrt_disc) / (2.0*A)

    lo, hi = sorted([mu2, mu1])
    cand = []
    if lo <= x1 <= hi: cand.append(x1)
    if lo <= x2 <= hi: cand.append(x2)
    if cand:
        mid = 0.5*(mu1+mu2)
        return min(cand, key=lambda x: abs(x-mid))
    mid = 0.5*(mu1+mu2)
    return x1 if abs(x1-mid) < abs(x2-mid) else x2

def _select_top3_by_weight(bgmm, labels):
    w = np.asarray(bgmm.weights_, float).ravel()
    k = w.size
    counts = np.array([(labels == i).sum() for i in range(k)], dtype=int)

    nonempty = np.where(counts > 0)[0]
    if nonempty.size > 0:
        idx = nonempty[np.argsort(w[nonempty])[::-1]].tolist()
    else:
        idx = []

    if len(idx) < 3:
        rest = [i for i in np.argsort(w)[::-1] if i not in idx]
        idx += rest[:(3 - len(idx))]
    return idx[:3]

def _compute_boundaries_for_selected(bgmm, sel_idx):
    mu = np.asarray(bgmm.means_, float).ravel()
    var = np.asarray(bgmm.covariances_, float).ravel()
    w   = np.asarray(bgmm.weights_, float).ravel()

    sel_mu = mu[sel_idx]
    sel_var= var[sel_idx]
    sel_w  = w[sel_idx]

    order = np.argsort(sel_mu)[::-1]
    mu_d  = sel_mu[order]
    var_d = sel_var[order]
    w_d   = sel_w[order]

    if mu_d.size >= 2:
        b12 = _quad_boundary(mu_d[0], var_d[0], w_d[0], mu_d[1], var_d[1], w_d[1])
    else:
        b12 = mu_d[0] if mu_d.size==1 else np.nan
    if mu_d.size >= 3:
        b23 = _quad_boundary(mu_d[1], var_d[1], w_d[1], mu_d[2], var_d[2], w_d[2])
    else:
        b23 = mu_d[-1] if mu_d.size>0 else np.nan

    bounds = np.array([b12, b23], float)
    bounds = bounds[np.isfinite(bounds)]
    if bounds.size == 0:
        bounds_log_desc = np.array([np.nan, np.nan], float)
    else:
        bounds_log_desc = np.sort(bounds)[::-1]

    means_lin = (10.0 ** mu_d).astype(float)
    means_lin_low2high = np.sort(means_lin)
    if means_lin_low2high.size < 3:
        padv = means_lin_low2high[-1] if means_lin_low2high.size>0 else np.nan
        means_lin_low2high = np.pad(means_lin_low2high, (0, 3-means_lin_low2high.size), constant_values=padv)

    if bounds_log_desc.size < 2:
        padv = bounds_log_desc[-1] if bounds_log_desc.size>0 else (np.nan if mu_d.size==0 else np.min(mu_d))
        bounds_log_desc = np.pad(bounds_log_desc, (0, 2-bounds_log_desc.size), constant_values=padv)

    return bounds_log_desc, mu_d, means_lin_low2high

### AIC

In [ ]:
# ===== figure =====
colors = ['tab:red', 'tab:blue', 'tab:grey', 'tab:grey']
gs = plt.GridSpec(1, 2, wspace=0, width_ratios=[1, 1])
fig = plt.figure(figsize=(2.5, 2.5))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)

# ===== MLE =====
log10_mle = _prep_log10_pos(AIC_MLE_ds, "AIC_MLE_ds")
bgmm_mle = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_mle[:, None])

labels_mle = bgmm_mle.predict(log10_mle[:, None])
mu_mle = bgmm_mle.means_.ravel()
var_mle = bgmm_mle.covariances_.ravel()
w_mle   = bgmm_mle.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_mle == i))
    print('weight:', w_mle[i])
    print('mu:', 10**mu_mle[i])
    print('var:', var_mle[i])

bins_lin = np.linspace(-3, 0.5, 35)
hist_mle, centers_mle, bw_mle = _safe_hist(log10_mle, bins_lin)
yvals_mle = 10**centers_mle
hscale_mle = bw_mle * 2.3 * yvals_mle

sel_idx_mle = _select_top3_by_weight(bgmm_mle, labels_mle)
bounds_mle_log_desc, sel_mu_mle_log10_desc, d_aicmle = _compute_boundaries_for_selected(bgmm_mle, sel_idx_mle)

b1_lin = 10**bounds_mle_log_desc[0]
b2_lin = 10**bounds_mle_log_desc[1]
b3_lin = yvals_mle[0]

globals()['bound1'] = b1_lin
globals()['bound2'] = b2_lin
globals()['aicMLEbound1'] = b1_lin
globals()['aicMLEbound2'] = b2_lin
globals()['aicMLEbound3'] = b3_lin

index1 = int(np.searchsorted(yvals_mle, b1_lin, side='left'))
index2 = int(np.searchsorted(yvals_mle, b2_lin, side='left'))
index3 = int(np.searchsorted(yvals_mle, b3_lin, side='left'))

ax1.barh(yvals_mle[index1:],            hist_mle[index1:],            height=hscale_mle[index1:],            color=colors[0], alpha=0.4)
ax1.barh(yvals_mle[index2:index1],      hist_mle[index2:index1],      height=hscale_mle[index2:index1],      color=colors[1], alpha=0.4)
ax1.barh(yvals_mle[index3:index2],      hist_mle[index3:index2],      height=hscale_mle[index3:index2],      color=colors[2], alpha=0.4)

ax1.invert_xaxis()
ax1.set_yscale('log')
ax1.set_ylim(0.0005, 5)
ax1.set_title('MLE')
ax1.tick_params(which='both', direction='in')

xspace2 = np.linspace(1, 0, 5)
for i in range(min(3, sel_mu_mle_log10_desc.size)):
    ax1.plot(xspace2, np.full_like(xspace2, 10**sel_mu_mle_log10_desc[i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    _seg_max(hist_mle, index1, None),
    _seg_max(hist_mle, index2, index1),
    _seg_max(hist_mle, index3, index2),
    _seg_max(hist_mle, None, index3),
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

# ===== MAP =====
log10_map = _prep_log10_pos(AIC_MAP_ds, "AIC_MAP_ds")
bgmm_map = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_map[:, None])

labels_map = bgmm_map.predict(log10_map[:, None])
mu_map = bgmm_map.means_.ravel()
var_map = bgmm_map.covariances_.ravel()
w_map   = bgmm_map.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_map == i))
    print('weight:', w_map[i])
    print('mu:', 10**mu_map[i])
    print('var:', var_map[i])

hist_map, centers_map, bw_map = _safe_hist(log10_map, bins_lin)
yvals_map = 10**centers_map
hscale_map = bw_map * 2.3 * yvals_map

sel_idx_map = _select_top3_by_weight(bgmm_map, labels_map)
bounds_map_log_desc, sel_mu_map_log10_desc, d_aicmap = _compute_boundaries_for_selected(bgmm_map, sel_idx_map)

b1m_lin = 10**bounds_map_log_desc[0]
b2m_lin = 10**bounds_map_log_desc[1]
b3m_lin = yvals_map[0]

globals()['aicMAPbound1'] = b1m_lin
globals()['aicMAPbound2'] = b2m_lin
globals()['aicMAPbound3'] = b3m_lin

index1_map = int(np.searchsorted(yvals_map, b1m_lin, side='left'))
index2_map = int(np.searchsorted(yvals_map, b2m_lin, side='left'))
index3_map = int(np.searchsorted(yvals_map, b3m_lin, side='left'))

ax2.barh(yvals_map[index1_map:],              hist_map[index1_map:],              height=hscale_map[index1_map:],              color=colors[0], alpha=0.4)
ax2.barh(yvals_map[index2_map:index1_map],    hist_map[index2_map:index1_map],    height=hscale_map[index2_map:index1_map],    color=colors[1], alpha=0.4)
ax2.barh(yvals_map[index3_map:index2_map],    hist_map[index3_map:index2_map],    height=hscale_map[index3_map:index2_map],    color=colors[2], alpha=0.4)

ax2.set_yscale('log')
ax2.yaxis.tick_right()
ax2.set_title('MAP')
ax2.set_xlabel('PDF')
ax2.xaxis.set_label_coords(0, -0.15)
ax2.tick_params(which='both', direction='in')

for i in range(min(3, sel_mu_map_log10_desc.size)):
    ax2.plot(xspace2, np.full_like(xspace2, 10**sel_mu_map_log10_desc[i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    hist_mle[index1:].max(initial=0),
    hist_mle[index2:index1].max(initial=0),
    hist_mle[index3:index2].max(initial=0),
    hist_mle[:index3].max(initial=0)
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

max_pdf_val_map = max(
    hist_map[index1_map:].max(initial=0),
    hist_map[index2_map:index1_map].max(initial=0),
    hist_map[index3_map:index2_map].max(initial=0)
)
ax2.set_xlim(0, max_pdf_val_map * 1.1)

plt.show()

print("d_aicmle (low→high):", d_aicmle)
print("d_aicmap (low→high):", d_aicmap)

### Bayesian

In [ ]:
# ===== figure =====
colors = ['tab:red', 'tab:blue', 'tab:grey', 'tab:grey']
gs = plt.GridSpec(1, 2, wspace=0, width_ratios=[1, 1])
fig = plt.figure(figsize=(2.5, 2.5))
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharey=ax1)

# ===== MLE =====
log10_mle = _prep_log10_pos(Bayes_MLE_ds, "Bayes_MLE_ds")
bgmm_mle = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_mle[:, None])

labels_mle = bgmm_mle.predict(log10_mle[:, None])
mu_mle = bgmm_mle.means_.ravel()
var_mle = bgmm_mle.covariances_.ravel()
w_mle   = bgmm_mle.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_mle == i))
    print('weight:', w_mle[i])
    print('mu:', 10**mu_mle[i])
    print('var:', var_mle[i])

bins_lin = np.linspace(-3, 0.5, 35)
hist_mle, centers_mle, bw_mle = _safe_hist(log10_mle, bins_lin)
yvals_mle = 10**centers_mle
hscale_mle = bw_mle * 2.3 * yvals_mle

sel_idx_mle = _select_top3_by_weight(bgmm_mle, labels_mle)
bounds_mle_log_desc, sel_mu_mle_log10_desc, d_baymle = _compute_boundaries_for_selected(bgmm_mle, sel_idx_mle)

b1_lin = 10**bounds_mle_log_desc[0]
b2_lin = 10**bounds_mle_log_desc[1]
b3_lin = yvals_mle[0]

globals()['bound1'] = b1_lin
globals()['bound2'] = b2_lin
globals()['bayesMLEbound1'] = b1_lin
globals()['bayesMLEbound2'] = b2_lin
globals()['bayesMLEbound3'] = b3_lin

index1 = int(np.searchsorted(yvals_mle, b1_lin, side='left'))
index2 = int(np.searchsorted(yvals_mle, b2_lin, side='left'))
index3 = int(np.searchsorted(yvals_mle, b3_lin, side='left'))  # 보통 0

ax1.barh(yvals_mle[index1:],            hist_mle[index1:],            height=hscale_mle[index1:],            color=colors[0], alpha=0.4)
ax1.barh(yvals_mle[index2:index1],      hist_mle[index2:index1],      height=hscale_mle[index2:index1],      color=colors[1], alpha=0.4)
ax1.barh(yvals_mle[index3:index2],      hist_mle[index3:index2],      height=hscale_mle[index3:index2],      color=colors[2], alpha=0.4)

ax1.invert_xaxis()
ax1.set_yscale('log')
ax1.set_ylim(0.0005, 5)
ax1.set_title('MLE')
ax1.tick_params(which='both', direction='in')

xspace2 = np.linspace(1, 0, 5)
for i in range(min(3, sel_mu_mle_log10_desc.size)):
    ax1.plot(xspace2, np.full_like(xspace2, 10**sel_mu_mle_log10_desc[i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    _seg_max(hist_mle, index1, None),
    _seg_max(hist_mle, index2, index1),
    _seg_max(hist_mle, index3, index2),
    _seg_max(hist_mle, None, index3),
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

# ===== MAP =====
log10_map = _prep_log10_pos(Bayes_MAP_ds, "Bayes_MAP_ds")
bgmm_map = BGMM(
    n_components=4, max_iter=50, n_init=20, init_params='kmeans',
    random_state=1, weight_concentration_prior=20, mean_precision_prior=0.5,
    covariance_type='spherical', covariance_prior=0.3, degrees_of_freedom_prior=5
).fit(log10_map[:, None])

labels_map = bgmm_map.predict(log10_map[:, None])
mu_map = bgmm_map.means_.ravel()
var_map = bgmm_map.covariances_.ravel()
w_map   = bgmm_map.weights_.ravel()

for i in range(4):
    print(np.count_nonzero(labels_map == i))
    print('weight:', w_map[i])
    print('mu:', 10**mu_map[i])
    print('var:', var_map[i])

hist_map, centers_map, bw_map = _safe_hist(log10_map, bins_lin)
yvals_map = 10**centers_map
hscale_map = bw_map * 2.3 * yvals_map

sel_idx_map = _select_top3_by_weight(bgmm_map, labels_map)
bounds_map_log_desc, sel_mu_map_log10_desc, d_baymap = _compute_boundaries_for_selected(bgmm_map, sel_idx_map)

b1m_lin = 10**bounds_map_log_desc[0]
b2m_lin = 10**bounds_map_log_desc[1]
b3m_lin = yvals_map[0]

globals()['bayesMAPbound1'] = b1m_lin
globals()['bayesMAPbound2'] = b2m_lin
globals()['bayesMAPbound3'] = b3m_lin

index1_map = int(np.searchsorted(yvals_map, b1m_lin, side='left'))
index2_map = int(np.searchsorted(yvals_map, b2m_lin, side='left'))
index3_map = int(np.searchsorted(yvals_map, b3m_lin, side='left'))

ax2.barh(yvals_map[index1_map:],              hist_map[index1_map:],              height=hscale_map[index1_map:],              color=colors[0], alpha=0.4)
ax2.barh(yvals_map[index2_map:index1_map],    hist_map[index2_map:index1_map],    height=hscale_map[index2_map:index1_map],    color=colors[1], alpha=0.4)
ax2.barh(yvals_map[index3_map:index2_map],    hist_map[index3_map:index2_map],    height=hscale_map[index3_map:index2_map],    color=colors[2], alpha=0.4)

ax2.set_yscale('log')
ax2.yaxis.tick_right()
ax2.set_title('MAP')
ax2.set_xlabel('PDF')
ax2.xaxis.set_label_coords(0, -0.15)
ax2.tick_params(which='both', direction='in')

for i in range(min(3, sel_mu_map_log10_desc.size)):
    ax2.plot(xspace2, np.full_like(xspace2, 10**sel_mu_map_log10_desc[i]), '--', linewidth=2, color=colors[i])

max_pdf_val_mle = max(
    hist_mle[index1:].max(initial=0),
    hist_mle[index2:index1].max(initial=0),
    hist_mle[index3:index2].max(initial=0),
    hist_mle[:index3].max(initial=0)
)
ax1.set_xlim(max_pdf_val_mle * 1.1, 0)

max_pdf_val_map = max(
    hist_map[index1_map:].max(initial=0),
    hist_map[index2_map:index1_map].max(initial=0),
    hist_map[index3_map:index2_map].max(initial=0)
)
ax2.set_xlim(0, max_pdf_val_map * 1.1)

plt.show()


print("d_baymle (low→high):", d_baymle)
print("d_baymap (low→high):", d_baymap)


In [ ]:
payload = {
    "version": 1,
    "created": datetime.now().isoformat(timespec="seconds"),
    "note": "Top-3 significant cluster means (linear, low->high)",
    "arrays": {
        "d_aicmle": d_aicmle.astype(float),
        "d_aicmap": d_aicmap.astype(float),
        "d_baymle": d_baymle.astype(float),
        "d_baymap": d_baymap.astype(float),
    }
}

save_path = "bgmm_top3_means.pkl"
with open(save_path, "wb") as f:
    pickle.dump(payload, f)

print(f"Saved: {save_path}")
for k, v in payload["arrays"].items():
    print(k, v)